In [1]:
%run ../../../utils/commonImports.py
%run ../../../utils/tradingImports.py
%matplotlib inline

from exputils import *

# Load data

In [2]:
# dropbox_dir = 'D:\\Dropbox\\My work\\krypl-project'
dropbox_dir = '/Users/david.vesely/Dropbox/My work/krypl-project'
features = ['support']
pairs = read_tsv('selected-pairs.tsv', header=None)[0].tolist()

data_dict = {p: read_all(p, dropbox_dir) for p in pairs}
supports = read_tsv('supports.tsv')

# Label supports

In [3]:
window_sizes = [4]

DAY = 48
max_hold_time = 5 * DAY
stop_loss = 0.05
fee = 0.0025

In [27]:
from tqdm import tqdm_notebook

def return_with_fee(price_s, price_e, fee):
    fee_part = (1 - fee) ** 2
    return ((price_e * fee_part) / price_s) - 1


def max_profit(data, index, next_index):
    index = int(index)
    next_index = len(data) if np.isnan(next_index) else int(next_index)
    if len(data) < index + 1:
        return None
    last = index + max_hold_time if len(data) > index + max_hold_time else len(data) - 1
    last = next_index if next_index < last else last
    selected = reset_index_hard(data.iloc[index+1:last, :].copy())
    buy_price = selected['close'].iloc[0]
    selected['_return'] = (selected['close'] / buy_price) - 1
    stop_losses = selected[selected['_return'] <= -stop_loss].reset_index()
    if len(stop_losses) > 0:
        stop_loss_i = stop_losses['index'].min()
        selected = selected.iloc[:stop_loss_i, :]
        
    return return_with_fee(buy_price, selected['high'].max(), fee)

n = len(pairs) * len(window_sizes)
with tqdm_notebook(total=n) as pbar:
    supports_data = pd.DataFrame()
    for pair in pairs:
        for win_size in window_sizes:
            data = data_dict[pair]
            supports_df = supports.query(f'pair == "{pair}"')\
                .query(f'win_size == {win_size}')
            
            support_indexes = supports_df['index']
            supports_df['max_profit'] = [
                max_profit(data, i, next_i) for i, next_i in zip(support_indexes, support_indexes.shift(-1))
            ]
            
            supports_data = supports_data.append(supports_df)
            pbar.update()

In [28]:
write_tsv(supports_data, 'labeled.tsv')

In [32]:
supports_df

,index,support,pair,win_size,date,max_profit
99330,32.0000,0.0001,BTC_XPM,4,2016-01-01 16:00:00,0.0055
99331,49.0000,0.0001,BTC_XPM,4,2016-01-02 00:30:00,0.0570
99332,70.0000,0.0001,BTC_XPM,4,2016-01-02 11:00:00,0.0218
99333,121.0000,0.0001,BTC_XPM,4,2016-01-03 12:30:00,-0.0050
99334,131.0000,0.0001,BTC_XPM,4,2016-01-03 17:30:00,0.0252
99335,145.0000,0.0001,BTC_XPM,4,2016-01-04 00:30:00,0.0286
99336,149.0000,0.0001,BTC_XPM,4,2016-01-04 02:30:00,0.0161
99337,171.0000,0.0001,BTC_XPM,4,2016-01-04 13:30:00,0.0180
99338,177.0000,0.0001,BTC_XPM,4,2016-01-04 16:30:00,-0.0050
99339,186.0000,0.0001,BTC_XPM,4,2016-01-04 21:00:00,0.0146
